In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [1]:
!pip install git+https://github.com//andreinechaev/nvcc4jupyter.git

  Cloning https://github.com//andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-jo3wu0bb
  Running command git clone --filter=blob:none --quiet https://github.com//andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-jo3wu0bb
  Resolved https://github.com//andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4293 sha256=743a10a16305de910383e4fe514bad2e86baf8a97c3c5e05ac9c0e3775706e2c
  Stored in directory: /tmp/pip-ephem-wheel-cache-s43l7c21/wheels/f4/70/5d/b3b10f451cc69546b4feecc0d4c76cf43cdb63b93ba62cfecb
Successfully built NVCCPlugin


In [27]:
%%writefile Quick_Sort.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

__device__ void quickSortKernel(int* array, int left, int right) {
    if (left < right) {
        int pivot = array[right];
        int partitionIndex = left;

        for (int i = left; i < right; ++i) {
            if (array[i] < pivot) {
                // Swap array[i] and array[partitionIndex]
                int temp = array[i];
                array[i] = array[partitionIndex];
                array[partitionIndex] = temp;
                partitionIndex++;
            }
        }

        // Swap pivot with array[partitionIndex]
        int temp = array[right];
        array[right] = array[partitionIndex];
        array[partitionIndex] = temp;

        // Recursively sort left and right partitions
        quickSortKernel(array, left, partitionIndex - 1);
        quickSortKernel(array, partitionIndex + 1, right);
    }
}


__global__ void quickSort(int* array, int left, int right) {
    if (left < right) {
        int pivot = array[right];
        int partitionIndex = left;

        for (int i = left; i < right; ++i) {
            if (array[i] < pivot) {
                // Swap array[i] and array[partitionIndex]
                int temp = array[i];
                array[i] = array[partitionIndex];
                array[partitionIndex] = temp;
                partitionIndex++;
            }
        }

        // Swap pivot with array[partitionIndex]
        int temp = array[right];
        array[right] = array[partitionIndex];
        array[partitionIndex] = temp;

        // Recursively sort left and right partitions
        quickSortKernel(array, left, partitionIndex - 1);
        quickSortKernel(array, partitionIndex + 1, right);
    }
}

int main() {
    const int n = 1000; // Size of the array
    int* array_h = (int*)malloc(sizeof(int) * n);

    // Generate random numbers and fill the array
    srand(time(NULL));
    for (int i = 0; i < n; ++i) {
        array_h[i] = rand() % 1000; // Generate random numbers between 0 and 999
    }

    // Print the unsorted array
    printf("Unsorted array:\n");
    for (int i = 0; i < n; ++i) {
        printf("%d ", array_h[i]);
    }
    printf("\n");

    int* array_dev;
    cudaMalloc((void**)&array_dev, n * sizeof(int));
    cudaMemcpy(array_dev, array_h, n * sizeof(int), cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    quickSort<<<1, 1>>>(array_dev, 0, n - 1);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    cudaMemcpy(array_h, array_dev, n * sizeof(int), cudaMemcpyDeviceToHost);

    // Print the sorted array
    printf("Sorted array:\n");
    for (int i = 0; i < n; ++i) {
        printf("%d ", array_h[i]);
    }
    printf("\n");

    printf("Runtime: %.3f milliseconds\n", milliseconds);

    // Clean up memory and other resources
    free(array_h);
    cudaFree(array_dev);

    return 0;
}

Overwriting aes_gcm_example.cu


In [31]:
!nvcc -o Quick_Sort Quick_Sort.cu 


In [32]:
!./Quick_Sort